In [ ]:
%matplotlib inline
import warnings  
warnings.filterwarnings('ignore')

import os
import json
import glob
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

In [ ]:
with open("../input/2022-ml-w9p2/mnist3D_data.json", 'r') as j:
    data = json.load(j)

In [ ]:
from torch.utils.data import Dataset

class MnistDataset(Dataset):
    def __init__(self, data, mode='train'):
        self.data = data[mode]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        xyz = np.array(self.data[idx]['points'], dtype=np.float32)
        return {'points':xyz, 'label':self.data[idx]['label']}

In [ ]:

def VoxelGrid(points, voxel_size=[1, 1, 1]):

    # Voxel Grid를 구하기 위해 각 축 별 최대, 최소값 계산
    min_xyz  = np.min(points, axis=0) - 0.001
    max_xyz  = np.max(points, axis=0) + 0.001

    diff = max(max_xyz - min_xyz ) - ( max_xyz - min_xyz )
    min_xyz  = min_xyz  - diff / 2
    max_xyz  = max_xyz  + diff / 2

    # voxel_grid
    voxel_grid = []

    # point cloud를 각 축 (x,y,z) 에 대한 voxel grid 로 분할
    for i in range(3):
        # Voxel_size는 정수만 가능
        if type(voxel_size[i]) is not int:
            raise TypeError("voxel_size[{}] must be int".format(i))
        segment = np.linspace( min_xyz[i], max_xyz[i], num=(voxel_size[i] + 1))
        voxel_grid.append(segment)

    return voxel_grid # 생성한 voxel_grid 를 반환

In [ ]:
def VoxelVector(points, voxel_grid, voxel_size=[1, 1, 1]):

    n_voxels = voxel_size[0] * voxel_size[1] * voxel_size[2]
    n_x = voxel_size[0]
    n_y = voxel_size[1]
    n_z = voxel_size[2]

    voxelization = np.zeros((len(points), 4), dtype=int)

    # Voxelization
    voxelization[:,0] = np.searchsorted(voxel_grid[0], points[:,0]) - 1
    voxelization[:,1] = np.searchsorted(voxel_grid[1], points[:,1]) - 1
    voxelization[:,2] = np.searchsorted(voxel_grid[2], points[:,2]) - 1

    voxelization[:,3] = ((voxelization[:,1] * n_x) + voxelization[:,0]) + (voxelization[:,2] * (n_x * n_y)) 

    # Voxelization으로부터 voxel_vector 추출
    vector = np.zeros(n_voxels)
    count = np.bincount(voxelization[:,3])
    vector[:len(count)] = count

    vector = vector.reshape(n_z, n_y, n_x)

    return vector # 추출한 Vector 반환

In [ ]:
def VoxelFeature(dataset):
    
    voxel_vectors = []
    labels = []

    for i, pc in enumerate(tqdm(dataset)):

        # point cloud를 np.array(dtype=np.float32)로 변경
        pointcloud = np.array(pc['points'], dtype = np.float32)   

        voxel_grid = VoxelGrid(pointcloud, voxel_size=[8, 8, 8])
        
        vector = VoxelVector(pointcloud, voxel_grid, voxel_size=[8, 8, 8])

        voxel_vectors.append(vector.reshape(-1) / np.max(vector))
        
        labels.append(pc['label'])

    return np.array(voxel_vectors), np.array(labels)

In [ ]:
mnist_train = MnistDataset(data, mode = "train")
mnist_test = MnistDataset(data, mode = "test")

x_train, y_train = VoxelFeature(mnist_train)
x_test, _ = VoxelFeature(mnist_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

model = SVC(kernel='poly')
model.fit(x_train_scale, y_train)
res = model.predict(x_test_scale)
res = pd.DataFrame(res, columns=['Label'])
res.to_csv('submit.csv', index_label='id')